## Neural Networkの典型的な手順
1. 学習可能なパラメータをもつネットワークを定義する。
1. 入力データセットを繰り返す。
1. ネットワークを介して、入力を処理する。
1. 正解からどれだけ遠いかを定義した損失を計算する。
1. 勾配をネットワークのパラメータに逆伝播させる。
1. ネットワークの重みを更新する。
    1. 以下のルールがよく使われる。weight = weight - learning_rate * gradient

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
#forwardさえ定義していれば、backwardは自動的に定義される。
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [10]:
#sizeが1, 1, 32, 32なのは、バッチサイズ、チャンネル数、x、yで構成されるから。必ず4次元tensorとしなくてはいけない。
#nSamples x nChannels x Height x Width.
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0296,  0.0578,  0.0169,  0.0854, -0.0165, -0.0029, -0.0638,  0.0009,
         -0.2010,  0.0098]], grad_fn=<AddmmBackward>)


In [11]:
#Zero the gradient buffers of all parameters and backprops with random gradients:
#この挙動がいまいちわからん。パラメータのgradient buffers？をゼロにし、ランダムな勾配を逆伝搬させる。
net.zero_grad()
out.backward(torch.randn(1, 10))

In [12]:
#loss function. Mean Squared Error
#criterionで、損失関数のインスタンスを作成する。それに対し、予測値と真値を入力し、誤差を計算する。
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.1172, grad_fn=<MseLossBackward>)


In [13]:
#loss.grad_fnで損失の計算順序を可視化することが可能。
"""
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
"""
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [14]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0097,  0.0007,  0.0032, -0.0091,  0.0107, -0.0139])


In [15]:
#pytorchには、最適化計算ライブラリがあるので、それを使いましょう。
import torch.optim as optim
# 最適化インスタンスの作成と学習率の設定
optimizer = optim.SGD(net.parameters(), lr=0.01)
# 基本的に、最初に、勾配をゼロにしておくのか？初期化？
optimizer.zero_grad()
# 入力をネットワークに入れる。
output = net(input)
# 損失の計算
loss = criterion(output, target)
# 逆伝播の実行
loss.backward()
# 最適化計算し、パラメータの更新
optimizer.step()